# Development: Integrate PIBT Deadlock Resolution in RCA

In [1]:
import sys
sys.path.append('../')

sys.path.append('../../')
import numpy as np
import cvxpy as cp
from matplotlib import pyplot as plt
from panav.environment.env import MultiTunnelEnv
from panav.hybrid import HybridGraph

from panav.viz import draw_env

import shapely

from panav.ORCA import Ordered_Agent

%load_ext autoreload
%autoreload 2

%load_ext snakeviz

In [116]:

bloating_r = 0.5
vmax = 1.0

N = 40


results = []

env_name = "MultiTunnel"


env = MultiTunnelEnv(n_tunnel = 4, tunnel_width = bloating_r*2.5,limits=[(-12,12),(-20,20)], N_agent = N)
HG = HybridGraph(env,bloating_r)

# For development purposes, force the soft edge weights to be zero, so that the agents strictly don't have head on collisions.

for e in HG.edges:
    if HG.edges[e]['type']=='soft':
        # HG.edges[e]['weight'] = 0
        pass


/Users/tianpengzhang/opt/miniconda3/envs/PA-Nav/lib/python3.11/site-packages/shapely/predicates.py:798: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)


In [117]:
from panav.TrafficAwarePlanning import traffic_aware_HG_plan
paths = traffic_aware_HG_plan(HG)

ref_plan = [np.array([HG.node_loc(u) for u in path]).T for path in paths]

In [118]:
plans = ref_plan

tau = 1.0 # The safe time interval. Can be generously long.
exec_tau = 0.95 * tau # Leaving a slight horizon margin helps avoid numerical inaccuracy in CVXPY optimization results.
agents = np.arange(N)
start_locs = env.starts
goal_locs = env.goals
# The execution time of ORCA velocity.
# Should be much shorter than the safe interval tau.

pos = []

v_prefs = [np.zeros(start_locs[0].shape) for a in agents]

protocol = 0

orcas = [Ordered_Agent(protocol,tau,bloating_r,vmax,p,init_v = None) 
         for p in start_locs]

# We will assume agent i is ranked i among the agents when dealing with conflicts.

curr_wp_index = [0 for a in agents]

def calc_pref(agent):

    wp = plans[agent][:,curr_wp_index[agent]]
    # See if the agent has reached the current waypoint.
    if np.linalg.norm(wp-orcas[agent].p)<= bloating_r:  
        curr_wp_index[agent] = \
        np.min([curr_wp_index[agent]+1,   
                plans[agent].shape[1]-1
                                ])
        
        wp = plans[agent][:,curr_wp_index[agent]]
        
    to_wp = wp-orcas[agent].p
    
    v_prefs[agent] = to_wp/tau if tau*vmax > np.linalg.norm(to_wp) else vmax *  to_wp/(np.linalg.norm(to_wp)+1e-5)

def PIBT(a):

    P = []
    C = []

    for nb in range(len(orcas)):
        # Find all agents in the that could collide with a in the next tau seconds.
        if nb!=a and np.linalg.norm(orcas[nb].p-orcas[a].p)<orcas[nb].bloating_r+orcas[a].bloating_r\
                                                  + 2 * vmax * tau:
            if np.linalg.norm(orcas[nb].p-orcas[a].p)<orcas[nb].bloating_r+orcas[a].bloating_r:
                print("Soft Collision. Agents ",a,nb,"Dist",np.linalg.norm(orcas[nb].p-orcas[a].p))
                raise Exception
            if orcas[nb].v is None:
                C.append(nb)
            else:
                P.append(nb)
            

    # print("Agent",a,"P",P,"C",C)

    candidate_v_pref = [] 

    for theta in np.pi * np.linspace(0,2,4)[:-1]:
            # Rotate v_pref clockwise by theta.
            v_right = np.array([[np.cos(-theta),-np.sin(-theta)],
                                [np.sin(-theta),np.cos(-theta)]]).dot(v_prefs[a])
            candidate_v_pref.append(v_right)

    candidate_v_pref.append(np.array([0,0])) # Always have zero velocity as a default 
    
    for i,v_pref in enumerate(candidate_v_pref):
        orcas[a].update_v(v_pref,env.obstacles,[orcas[b] for b in P]) 
        if orcas[a].v is None:
            # print("Agent",a,"infeasible")
            return False
        
        print("Agent",a,"v_pref",i,"feasible",'v',orcas[a].v,'v_opt',orcas[a].v_opt)
        
        children_valid = True
        for c in C:
            if orcas[c].v is None:
                print("Parent agent",a," calling PIBT for agent",c,"in recursive call")
                children_valid = PIBT(c)
                if not children_valid:
                    break
        
        if children_valid:
            return True

    orcas[a].v = None
    return False

for _ in range(200):
    print("################### Time step {} ################".format(_))
    pos.append(np.array([a.p for a in orcas]))
       
    for a in agents:
        # print("Agent",a)

        # Compute the preferred velocity.
        calc_pref(a)  


        
        
    
    for a in agents:
        if orcas[a].v is None:
            print("PIBT for agent",a,"in outer loop")
            valid = PIBT(a)
            print('Agent',a,'Valid',valid," in outer loop")
    
    # We assume agent a is ranked a among the agents when dealing with conflicts.
    
    # Execute the safe velocity.
    all_reached = True
    
    for a in agents:
        dist2goal = np.linalg.norm(orcas[a].p - goal_locs[a])
        # print(a,dist2goal)

        if dist2goal>=1*bloating_r:
            orcas[a].p += orcas[a].v*exec_tau
            all_reached = False
            # Reset all agent's v to be None
            orcas[a].v = None
        else:
            orcas[a].v = orcas[a].v_opt = np.array([0,0])
    if all_reached:
        break

################### Time step 0 ################
################### Time step 1 ################
PIBT for agent 0 in outer loop
Agent 0 v_pref 0 feasible v [-0.43696 -0.89948] v_opt [-0.43696 -0.89948]
Parent agent 0  calling PIBT for agent 1 in recursive call
Agent 1 v_pref 0 feasible v [-0.49262 -0.87024] v_opt [-0.49262 -0.87024]
Parent agent 1  calling PIBT for agent 2 in recursive call
Agent 2 v_pref 0 feasible v [-0.32552 -0.94554] v_opt [-0.32552 -0.94554]
Parent agent 2  calling PIBT for agent 3 in recursive call
Agent 3 v_pref 0 feasible v [-0.64568 -0.76361] v_opt [-0.64568 -0.76361]
Parent agent 3  calling PIBT for agent 4 in recursive call
Agent 4 v_pref 0 feasible v [-0.74674 -0.66511] v_opt [-0.74674 -0.66511]
Parent agent 4  calling PIBT for agent 5 in recursive call
Agent 5 v_pref 0 feasible v [-0.85797 -0.5137 ] v_opt [-0.85797 -0.5137 ]
Parent agent 5  calling PIBT for agent 6 in recursive call
Agent 6 v_pref 0 feasible v [-0.95602 -0.29329] v_opt [-0.95602 -0.29329]

KeyboardInterrupt: 

In [115]:
from panav.viz import animation
from IPython.display import HTML, display
pos = np.array(pos)

dt = exec_tau

pos_to_ani = [pos[:,a,:].T for a in agents]
anim = animation(env, pos_to_ani, bloating_r, dt = dt)
display(HTML(anim.to_jshtml()))
plt.close() 